In [1]:
import AlGDock.Nanopore
from AlGDock.Nanopore import *

self = AlGDock.Nanopore.Simulation(\
  ligand_tarball='prmtopcrd/ligand.tar.gz', \
  ligand_database='ligand.db', \
  forcefield='prmtopcrd/gaff.dat', \
  frcmodList=['ligand.frcmod'], \
  ligand_prmtop='ligand.prmtop', \
  grid_LJr='grids/LJr.nc', \
  starting_conf='prmtopcrd/anchor_and_grow_scored.mol2', \
  ef=1.0E8, \
  max_trials=10000, \
  report_interval=100)

  extracted /Users/dminh/Installers/AlGDock-0.0.1/Nanopore_Example/ligand.db
  extracted /Users/dminh/Installers/AlGDock-0.0.1/Nanopore_Example/ligand.frcmod
  extracted /Users/dminh/Installers/AlGDock-0.0.1/Nanopore_Example/ligand.prmtop
  grid type LJa not found

         /Users/dminh/Installers/AlGDock-0.0.1/Nanopore_Example/ligand.db,
         using first one
/Users/dminh/Installers/AlGDock-0.0.1/Nanopore_Example/ligand.db
/Users/dminh/Installers/AlGDock-0.0.1/Nanopore_Example/ligand.db



  grid type ELE not found


In [2]:
self.run()

Simulating motion from 0.20000 to 1.80000


/Users/dminh/Applications/miniconda2/envs/algdock/lib/python2.7/site-packages/simtk/openmm/app/internal/amber_file_parser.py:1010: UserWarning: Non-optimal GB parameters detected for GB model OBC2
  'model %s' % gbmodel)


Trial      0, COM: [0.4964267589425149, 0.4964840328192567, 0.24038602825216784]
Trial    100, COM: [0.44850141721871634, 0.6748687932696901, 0.3698771411710205]
Trial    200, COM: [0.39458045026003524, 0.5999999869968635, 0.639199867954873]
Trial    300, COM: [0.4113949691637279, 0.5146116315738214, 1.0097857006843052]
Trial    400, COM: [0.546017894305857, 0.4230026543530237, 1.108873006598246]
Trial    500, COM: [0.48144303321318377, 0.3532859781963921, 1.3537315839037514]
Trial    600, COM: [0.4572700451417212, 0.4559887072073787, 1.4758613694009635]
Trial    700, COM: [0.3193484138461007, 0.49833354427079884, 1.6351348844160674]
Trial    800, COM: [0.3566156347175168, 0.5503431103637217, 1.676988563950804]
Trial    900, COM: [0.2837418224959771, 0.5575837676460137, 1.5963611918578295]
Trial   1000, COM: [0.365783967241433, 0.5513384259842201, 1.6282082214202067]
Trial   1100, COM: [0.502321132175905, 0.42152714046866363, 1.660508267319922]
Trial   1200, COM: [0.5244481614669858, 0

In [3]:
self.view_trajectory()